# Theoretical Analysis
> Theoretical Analysis (level 0) for all CNN topologies and hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [Rooflines,MNIST,ImageNet,CIFAR-10]
- image: images/roofline.png

In [1]:
#hide
import numpy as np
import pandas as pd
import random
import re

pd.options.display.max_rows = 10000 # this will set limit of columns to 500
pd.options.display.float_format = '{:20,.4f}'.format
import altair as alt
W = 600
H = 480

In [2]:
#hide
#notes
#rn50_df = df[df.apply(lambda row: row['net_prun_datatype'].split('_')[0] == 'MobileNetv1|ResNet50', axis=1)]
#rn50_df['net_prun_datatype'] = rn50_df['net_prun_datatype'].str.split('_').str[0] #comment this when I want to hide the topology
#rn50_df['hardw'] = rn50_df['hardw'].str.replace("_INT8|_FP16|_FP32", "")

# Introduction

This page presents a Theoretical Analysis of both hardware platforms as well as CNN topologies.
In order to get a general overview of all CNNs and Hardware Platforms included in our experiments, we present the following 3 tables. 

# Tables

### CNNs and Their Accuracy Over All Pruning and Quantization Variants

Table below provides a complete overview of all CNNs that were included in the experimentation and their corresponding accuracy over all Pruning and Quantization Variants.

In [3]:
#hide_input
%run scripts/script_tables.py  #run the tables script
tableOverviewExperiments(['data/cnn_topologies_accuracy.csv'])

,INT2,INT4,INT8,FP16,FP32
,top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%],top1 (top5) [%]
GoogLeNetv1,nm,nm,69.24 (88.45),66.93 (87.83),66.96 (87.84)
MobileNetv1,nm,nm,69.57 (87.71),nm,nm
EfficientNet-S 100%,nm,nm,77,nm,nm
EfficientNet-M 100%,nm,nm,78.6,nm,nm
EfficientNet-L 100%,nm,nm,80.2,nm,nm
ResNet50 100%,nm,nm,73.29 (91.26),75.14 (92.12),75.15 (92.11)
ResNet50 80%,nm,nm,73.30 (91.40),nm,nm
ResNet50 50%,nm,nm,69.49 (91.00),nm,nm
ResNet50 30%,nm,nm,68.83 ( 90.16),nm,nm


### CNNs and Their Compute and Memory Requirements

Next table shows Compute and Memory Requirements for all CNNs in number of operations ([GOPs]), Model Size ([ME]) and Operational Intensity ([OI]) in operations per byte read or written from memory.

In [4]:
#hide_input
#%run scripts/script_tables.py  #run the tables script if it hasn't been run
tableOverviewExperiments(['data/cnn_topologies_compute_memory_requirements.csv'])

,Total OPs,Total Model Size,OI (INT2),OI (INT4),OI (INT8),OI (FP16),OI (FP32)
,GOPs,[ME],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte],[Ops/Byte]
GoogLeNetv1,3.1,6,2093.97,1046.99,523.49,261.75,130.87
MobileNetv1,1.1,4.2,1075.47,537.74,268.87,134.43,67.22
ResNet-50 100%,7.7,25.5,1210.84,605.42,302.71,151.36,75.68
ResNet-50 80%,6.5,23.7,1086.59,543.3,271.65,135.82,67.91
ResNet-50 50%,3.8,15.8,949.85,474.93,237.46,118.73,59.37
ResNet-50 30%,2.5,10.1,970.16,485.08,242.54,121.27,60.64
EfficientNet Edge S,4.7,5.4,3481.48,1740.74,870.37,435.18,217.59
EfficientNet Edge M,7.4,6.9,4289.86,2144.93,1072.46,536.23,268.12
EfficientNet Edge L,19.4,10.6,7313.21,3656.6,1828.3,914.15,457.08


We created bar charts to better illustrate compute and memory requirements for all CNNs from the previous table in an interactive and easy way. 

In [5]:
#hide_input
%run scripts/altair_plots # sun the script which has the function to create the bar charts - if it hasn't been run yet
get_compute_memory_cnn_chart(csv_file= 'data/cnn_topologies_compute_memory_requirements.csv') 

alt.HConcatChart(...)

## Hardware Platforms

Table below summarizes all included hardware platforms, each with its corresponding peak performance for diferent datatypes (INTx, FPx), its Memory Bandwidth, Memory capacity as well as Thermal Design Power.

In [6]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/hardware_platforms.csv'])

Hardware Platforms,INT2,INT4,INT8,FP16,FP32,Memory Bandwidth,Memory Capacity,Power
,[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[TOP/sec],[GBps],[GB],[Watt]
Ultra96-DPU,na,na,0.96,na,na,4.26,2,na
ZCU104-DPU,na,na,4.6,na,na,19.2,4,na
ZCU102-DPU,na,na,6.71,na,na,19.2,4,na
ZCU104-FINN,30.7,8.8,na,na,na,19.2,4,na
ZCU104-BISMO,30.7,8.8,na,na,na,19.2,4,na
TX2 - maxn,na,na,na,1.33,0.67,59.7,8,15
TX2 - maxp,na,na,na,1.15,0.57,59.7,8,15
TX2 - maxq,na,na,na,0.87,0.44,59.7,8,15
EdgeTPU-fast,na,na,4,na,na,25.6,1,2


To better illustrate Hardware Platforms' Peak Performance and Memory Bandwidth, an interactive Bar chart can be found below.
Please note, only performance for natively supported datatypes are shown.

In [7]:
#hide_input
%run scripts/altair_plots.py # run the script if it hasn't been run before
get_peak_perf_bar_chart('data/hardware_platforms.csv')

alt.FacetChart(...)

## Overview of Theoretical Evaluation

link to: https://rcl-lab.github.io/Qutibench_Web/mnist/imagenet/cifar-10/2020/04/30/Overview_of_experiments.html

# Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [55]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe
    Given a dataframe and a specific column, this method replaces a string for another, both from the list of tuples
       
    Parameters
    ----------
     df_: pd.DataFrame()
        Dataframe with data to be replaced.
    column: str
        Column whithin dataframe where all replacements will take place.
    list_tuples_data_to_replace: list
        List with tuples which will contain what to replace by what.
        Eg.:list_tuples_data_to_replace = [(a,b), (c,d), (...) ] -> 'a' will be replaced by 'b', 'c' will be replaced by 'd', and so on.

    Returns
    -------
    df_out: pd.DataFrame()
       Dataframe with all indicated values replaced.
        
    """
    df_out = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df_out[column] = df_out[column].str.replace(j, k)
    return df_out
# Checkboxes with on-plot tooltips
def line_chart_w_checkbox(data: pd.DataFrame, condition: dict, selection: alt.vegalite.v4.api.Selection)->alt.vegalite.v4.api.Chart:
    """
    This function creates an Altair line chart with checkboxes.
    
    Parameters
    ----------
        data: pd.DataFrame
            Dataframe from which the plot will be created.       
        condition: dict
            Condition for the color.
            Eg.: {'condition': {'selection': 'FPGAs  Ultra96  DPU  ZCU  ', 'type': 'nominal', 'field': 'Name'}, 'value': 'lightgray'}      
        selection: Selection
            Selection object to select what information the checkbox is tied to.
            Eg.: Selection('FPGAs  Ultra96  DPU  ZCU  ', SelectionDef({
                             bind: BindCheckbox({ input: 'checkbox' }),
                             fields: ['Hide'],
                             type: 'single'
                          }))
    Returns
    -------
        Line Chart with checkboxes.          
    """
    maxX=160000
    width =700 
    height = 500
    chart = alt.Chart(data, width=width,height=height).properties(title='Comparing Hardware Platforms Rooflines and Neural Networks Arithmetic Intensity').mark_line(clip=True).encode(
        alt.X('arith_intens:Q', 
              title = 'ARITHMETIC INTENSITY (OPS/BYTE)', 
              scale = alt.Scale(type='log', domain = (0.1,maxX) )
             ),
        alt.Y('performance:Q', 
              title = 'PERFORMANCE (TOPS/S)', 
              scale=alt.Scale(type='log', domain = (0.2,40) )
             ),    
        color=condition
    ).add_selection(selection)
    return chart



def rooflines(dataframe: pd.DataFrame, neural_network: str)->alt.vegalite.v4.api.Chart:
    """
    This function creates an Altair line chart with checkboxes. Creates a lot of them and then sums them up.
    
    Parameters
    ----------
        data: pd.DataFrame
            Dataframe from which the plot will be created.       
        neural_network:str
            neural network that will also be plotted besides all hardware platforms.
            Eg.:'imagenet|mnist|cifar'
                'imagenet'
     Returns
    -------
        Line Chart with checkboxes, all charts are summed up.          
    """
    #hide_input
    maxX=160000
    width =700 
    height = 500
    data=dataframe
    
    hw_df   = dataframe[dataframe['Name'].str.contains("Ultra96|ZCU|TX2|TPU|NCS|A53")]
    
    #to select data to be plotted according to user input
    if neural_network in 'imagenet':
        nn_df   = dataframe[dataframe['Name'].str.contains("GoogLeNetv1|MobileNetv1|ResNet|EfficientNet")]
    elif neural_network in 'cifar':
        nn_df   = dataframe[dataframe['Name'].str.contains("CNV")]
    elif neural_network in 'mnist':
        nn_df   = dataframe[dataframe['Name'].str.contains("MLP")]
    elif neural_network in 'imagenet|mnist|cifar':
        nn_df   = dataframe[dataframe['Name'].str.contains("GoogLeNetv1|MobileNetv1|ResNet|EfficientNet|CNV|MLP")]
    else:
         return 'There were no results for the neural network asked. Please insert another network'
    
    #nn_df['Name'] = [re.sub(r'^MLP','MLPsss',str(x)) for x in nn_df['Name']]
    nn_df['Name']=nn_df['Name'].replace(r'.*MLP.*', 'MLP', regex=True)

    return nn_df
    #This part is to create all plots binded to checkboes-------------
    #Selecting data for each checkbox, from dataset. Each checkbox will be tied to each one of these data        
    FPGA_data   = dataframe[dataframe['Name'].str.contains("Ultra96 DPU|ZCU")]
    NVIDIA_data = dataframe[dataframe['Name'].str.contains("TX2")]
    GOOGLE_data = dataframe[dataframe['Name'].str.contains("EdgeTPU")]
    INTEL_data  = dataframe[dataframe['Name'].str.contains("NCS")]

    INT2_data = nn_df[nn_df['Name'].str.contains("INT2")]
    INT4_data    = nn_df[nn_df['Name'].str.contains("INT4")]
    INT8_data    = nn_df[nn_df['Name'].str.contains("INT8")]
    FP16_data = nn_df[nn_df['Name'].str.contains("FP16")]
    FP32_data     = nn_df[nn_df['Name'].str.contains("FP32")]

    #To say that the binding type will be a checkbox
    #BindCheckbox({ input: 'checkbox'})
    filter_checkbox = alt.binding_checkbox()

    #To create all checkboxes with the specifications info for each set
    #Selection('FPGAs:', SelectionDef({ bind: BindCheckbox({ input: 'checkbox' }), fields: ['Ultra96 DPU,ZCU104,ZCU102,ZCU104 FINN,ZCU104 BISMO'], type: 'single' }))
    FPGA_select   = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="FPGAs  Ultra96  DPU  ZCU  ")                 
    NVIDIA_select = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="INVIDIA  TX2  maxn, maxp, maxq  ")
    GOOGLE_select = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="GOOGLE  EdgeTPU, fast, slow  ")
    INTEL_select  = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="INTEL  NCS  ")

    INT2_select = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="INT2")
    INT4_select    = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="INT4")   
    INT8_select    = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="INT8")   
    FP16_select = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="FP16")
    FP32_select     = alt.selection_single( fields=["Hide"], bind=filter_checkbox, name="FP32")

    #Color Condiotions for each plot
    #{'condition': {'selection': 'FPGAs:', 'type': 'nominal', 'field': 'Name'}, 'value': 'lightgray'}
    FPGA_cond     = alt.condition(FPGA_select, alt.Color("Name:N"), alt.value("lightgray"))
    NVIDIA_cond   = alt.condition(NVIDIA_select, alt.Color("Name:N"), alt.value("lightgray"))
    GOOGLE_cond   = alt.condition(GOOGLE_select, alt.Color("Name:N"), alt.value("lightgray"))
    INTEL_cond    = alt.condition(INTEL_select, alt.Color("Name:N"), alt.value("lightgray"))

    INT2_cond = alt.condition(INT2_select, alt.Color("Name:N"), alt.value("lightgray"))
    INT4_cond    = alt.condition(INT4_select, alt.Color("Name:N"), alt.value("lightgray"))
    INT8_cond    = alt.condition(INT8_select, alt.Color("Name:N"), alt.value("lightgray"))
    FP16_cond = alt.condition(FP16_select, alt.Color("Name:N"), alt.value("lightgray"))
    FP32_cond     = alt.condition(FP32_select, alt.Color("Name:N"), alt.value("lightgray"))

    #Creating all plots 
    
    FPGA_chart     = line_chart_w_checkbox(FPGA_data,     FPGA_cond,    FPGA_select)
    NVIDIA_chart   = line_chart_w_checkbox(NVIDIA_data,   NVIDIA_cond,  NVIDIA_select)
    GOOGLE_chart   = line_chart_w_checkbox(GOOGLE_data,   GOOGLE_cond,  GOOGLE_select)                         
    INTEL_chart    = line_chart_w_checkbox(INTEL_data,    INTEL_cond,   INTEL_select)

    INT2_chart =    line_chart_w_checkbox(INT2_data, INT2_cond, INT2_select)
    INT4_chart    = line_chart_w_checkbox(INT4_data,    INT4_cond,    INT4_select)
    INT8_chart    = line_chart_w_checkbox(INT8_data,    INT8_cond,    INT8_select)
    FP16_chart =    line_chart_w_checkbox(FP16_data, FP16_cond, FP16_select)
    FP32_chart     = line_chart_w_checkbox(FP32_data,     FP32_cond,     FP32_select)

   
    #--------------------------------------------------------------------------------------------------
    # Create line plot
    line_chart = alt.Chart().mark_line(clip=True).interactive().encode(
            alt.X('arith_intens:Q'), 
            alt.Y('performance:Q'),
            alt.Color('Name:N', legend=alt.Legend(columns=2))
    )
    
        #Create the selection which chooses nearest point on mouse hoover
    selection = alt.selection(type='single', nearest=True, on='mouseover', fields=['arith_intens']) #to leave suggestions on, just replace arith_intens wiith anything else
    #Create text plot to show the text values on mouse hoovering
    text = (line_chart).mark_text(align='left', dx=3, dy=-3,clip=True).encode(  text=alt.condition(selection, 'Name:N', alt.value(' ')))


    #Creates the points plot for the NNs. The points will be invisible
    selectors = alt.Chart().mark_point(clip=True).encode(
                alt.X('arith_intens:Q'), 
                alt.Y('performance:Q'),
                opacity=alt.value(0),
    ).add_selection(selection)
    
    chart_all = (pd.Series([INT2_chart, INT4_chart, INT8_chart, FP16_chart, FP32_chart], name="charts")).to_frame()
   
    #Chart = alt.layer(FPGA_chart + NVIDIA_chart + GOOGLE_chart + INTEL_chart + INT2_chart + INT4_chart + INT8_chart + FP16_chart+ FP32_chart
    #Chart = alt.layer(chart_filtered.squeeze() + FPGA_chart + NVIDIA_chart + GOOGLE_chart + INTEL_chart, selectors, text, data=dataframe, width=700, height=500)
    Chart = alt.layer(chart_all.charts.sum(numeric_only = False) + FPGA_chart + NVIDIA_chart + GOOGLE_chart + INTEL_chart, selectors, text, data=pd.concat([nn_df, hw_df]), width=700, height=500)

    return Chart



In [56]:
#hide_input

#first process the following csv's to get clean ready to plot csv's
#%run scripts/script_load_save_data.py
#clean_csv_rooflines(path_topologies='data/cnn_topologies_compute_memory_requirements.csv',
#                    path_hardware='data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
#%run scripts/altair_plots.py
rooflines(pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 'imagenet|mnist|cifar')




C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,arith_intens,performance
51,GoogLeNetv1_INT2,"2,093.97",30.00
52,MobileNetv1_INT2,"1,075.47",30.00
53,ResNet-50 100%_INT2,"1,210.84",30.00
54,ResNet-50 80%_INT2,"1,086.59",30.00
55,ResNet-50 50%_INT2,949.85,30.00
56,ResNet-50 30%_INT2,970.16,30.00
57,EfficientNet Edge S_INT2,"3,481.48",30.00
58,EfficientNet Edge M_INT2,"4,289.86",30.00
59,EfficientNet Edge L_INT2,"7,313.21",30.00
60,CNV 100%_INT2,304.95,30.00


# Performance Prediction

The following heatmaps show the theoretical performance for the listed hardware platforms across the various machine learning tasks: MNIST, ImageNet and CIFAR-10. The metric used for the theoretical performance is input/second.

In [9]:
#hide
# First process the unfiltered csv and save it as a pretty csv ready to plotted as a heatmap
#%run scripts/script_load_save_data.py
#have to change clean_csv_performance_predictions to process the right csv the right way
#clean_csv_performance_predictions('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/performance_predictions_imagenet_mnist_cifar.csv')

### MNIST

For MNIST, quantization combined with pruning deliver some of best performance results.

In [10]:
#hide_input
%run scripts/altair_plots.py
#load mnist dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_mnist.csv"), 'red', 'Performance Prediction for MNIST')

alt.LayerChart(...)

### ImageNet

For ImageNet, quantization combined with pruning also deliver some of best performance results.

In [11]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load imagenet dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_imagenet.csv"), 'lightgrey','Performance Prediction for Imagenet')

alt.LayerChart(...)

### CIFAR-10

Finally, for CIFAR-10, quantization combined with pruning deliver some of best performance results

In [12]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/cleaned_csv/performance_prediction_cifar10.csv"), 'pink','Performance Prediction for CIFAR-10')

alt.LayerChart(...)

# Theoretical Pareto Curves

In the following plots we present a theoretical pareto curve for each type of classification. 

### MNIST

In [2]:
#hide_input
%run scripts/overlapped_pareto.py
theor_pareto(net_keyword='mnist', title= 'Theoretical Pareto for MNIST')

alt.LayerChart(...)

### ImageNet

In [1]:
#hide_input
%run scripts/overlapped_pareto.py
theor_pareto(net_keyword='imagenet', title= 'Theoretical Pareto for ImageNet')

alt.LayerChart(...)

### CIFAR-10

In [15]:
#hide_input
%run scripts/overlapped_pareto.py
theor_pareto(net_keyword='cifar', title= 'Theoretical Pareto for CIFAR-10')

alt.LayerChart(...)